In [18]:
import numpy as np
import string
from PIL import Image, ImageFont, ImageDraw
import random
import os
import cv2
import fnmatch
import cv2
import numpy as np
import string
import time

from keras_preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
from keras.activations import relu, sigmoid, softmax
import keras.backend as K
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os
import tensorflow as tf
import random
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.utils import shuffle
import pandas as pd

df = pd.read_csv('annotation.txt', sep=',') 

ParserError: Error tokenizing data. C error: Expected 2 fields in line 4, saw 3


In [2]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.1, random_state=42)


In [3]:
len(train)

8999

In [4]:
len(val)

1000

In [20]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Параметры
images_folder = 'images'  # папка с исходными изображениями
annotation_file = 'annotation.txt'  # файл с аннотациями
train_ratio = 0.8  # доля обучающей выборки
random_seed = 42  # для воспроизводимости результатов

# Создаем папки для train и test
os.makedirs('train/images', exist_ok=True)
os.makedirs('test/images', exist_ok=True)

# Читаем аннотации
with open(annotation_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Парсим данные
data = []
for line in lines:
    filename, label = line.strip().split(',', 1)
    data.append((filename, label))

# Разделяем данные
train_data, test_data = train_test_split(
    data, 
    train_size=train_ratio, 
    random_state=random_seed,
    shuffle=True
)

# Функция для сохранения данных
def save_dataset(data, dataset_name):
    image_folder = f'{dataset_name}/images'
    ann_file = f'{dataset_name}/annotation.txt'
    
    with open(ann_file, 'w', encoding='utf-8') as f:
        for filename, label in data:
            # Копируем изображение
            src_path = os.path.join(images_folder, filename)
            dst_path = os.path.join(image_folder, filename)
            shutil.copy2(src_path, dst_path)
            # Записываем аннотацию
            f.write(f'{filename},{label}\n')

# Сохраняем обе выборки
save_dataset(train_data, 'train')
save_dataset(test_data, 'test')

print(f'Распределение завершено!')
print(f'Обучающая выборка: {len(train_data)} примеров')
print(f'Тестовая выборка: {len(test_data)} примеров')

Распределение завершено!
Обучающая выборка: 8000 примеров
Тестовая выборка: 2000 примеров


In [7]:
import os
import torch.backends.cudnn as cudnn
import yaml
from train import train
from utils import AttrDict
import pandas as pd

cudnn.benchmark = True
cudnn.deterministic = False

def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

#Запускаем обучение
opt = get_config("config/custom_data_train.yaml")
train(opt, amp=False)

ModuleNotFoundError: No module named 'train'

In [17]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

# Конфигурация
images_dir = "images"  # папка с изображениями
labels_file = "labels.csv"  # файл с метками
train_dir = "train"  # папка для тренировочных данных
test_dir = "test"    # папка для тестовых данных
test_size = 0.2      # доля тестовой выборки
random_state = 42    # для воспроизводимости результатов

# Создаем папки для тренировочных и тестовых данных
os.makedirs(os.path.join(train_dir, "images"), exist_ok=True)
os.makedirs(os.path.join(test_dir, "images"), exist_ok=True)

# Загружаем метки
df = pd.read_csv(labels_file, header=None, names=["file", "label"])

# Разделяем данные
train_df, test_df = train_test_split(
    df, 
    test_size=test_size, 
    random_state=random_state,
    shuffle=True
)

# Функция для копирования файлов и создания меток
def process_dataset(df, target_dir):
    # Копируем изображения
    for file_name in df["file"]:
        src = os.path.join(images_dir, file_name)
        dst = os.path.join(target_dir, "images", file_name)
        shutil.copy2(src, dst)
    
    # Сохраняем метки
    labels_path = os.path.join(target_dir, "labels.csv")
    df.to_csv(labels_path, index=False, header=False)

# Обрабатываем обе выборки
process_dataset(train_df, train_dir)
process_dataset(test_df, test_dir)

print("Разделение завершено!")
print(f"Тренировочных samples: {len(train_df)}")
print(f"Тестовых samples: {len(test_df)}")

OSError: [WinError 123] Синтаксическая ошибка в имени файла, имени папки или метке тома

In [14]:
import os
import shutil
import csv
import random
from sklearn.model_selection import train_test_split

def split_dataset(images_dir, csv_file, output_dir, test_size=0.2, random_state=42):
    # Чтение данных из CSV
    data = []
    with open(csv_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:  # Проверяем, что в строке есть хотя бы 2 элемента
                data.append((row[0], row[1]))
    
    # Разделение на train и test
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=random_state)
    
    # Создание директорий
    train_dir = os.path.join(output_dir, 'train')
    test_dir = os.path.join(output_dir, 'test')
    
    os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
    os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
    
    # Функция для копирования файлов и создания CSV
    def process_dataset(data, dataset_dir):
        # Копируем изображения
        for img_name, label in data:
            src_path = os.path.join(images_dir, img_name)
            dst_path = os.path.join(dataset_dir, 'images', img_name)
            if os.path.exists(src_path):
                shutil.copy2(src_path, dst_path)
        
        # Создаем CSV файл
        csv_path = os.path.join(dataset_dir, 'output.csv')
        with open(csv_path, 'w', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            for img_name, label in data:
                writer.writerow([img_name, label])
    
    # Обрабатываем train и test наборы
    process_dataset(train_data, train_dir)
    process_dataset(test_data, test_dir)
    
    print(f"Данные разделены:")
    print(f"  Train: {len(train_data)} записей")
    print(f"  Test: {len(test_data)} записей")
    print(f"  Train данные сохранены в: {train_dir}")
    print(f"  Test данные сохранены в: {test_dir}")

# Использование
if __name__ == "__main__":
    # Укажите пути к вашим данным
    images_directory = "images/"  # Замените на реальный путь
    csv_file_path = "labels.csv"  # Замените на реальный путь
    output_directory = "outputttttt"  # Папка для результатов
    
    # Выполняем разделение
    split_dataset(images_directory, csv_file_path, output_directory)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [16]:
df = pd.read_csv("labels.csv")
df.head(4)

,"0.jpg,фБөbXҐДЌLKОp|Nt"
0,"1.jpg,TЊ5ЃF@ӯх{Г"
1,"2.jpg,_%BЌc"
2,"3.jpg,;ЕвvУеџ/й,Zџё>5Ш"
3,"4.jpg,gљYс3I"


In [21]:
df3 = pd.read_csv("train.csv", sep = ',')

In [22]:
df3

,filenames,words
0,0.jpg,5Hвkг
1,1.jpg,ӢWвйДfsЯћ
2,2.jpg,?;ГoJю'/ФЩЁ
3,3.jpg,}n&0oZаҚКZUBрѓ}cEnмS
4,4.jpg,Ш)ҚLЯ
...,...,...
7995,7995.jpg,эGv9mD@Қfша:т
7996,7996.jpg,ЖЁШiМ|љЋV
7997,7997.jpg,KйQЈ5пчirGLќyмйяЮ€
7998,7998.jpg,4*кРїG&Ша|оҳы+ђ>Ш


In [2]:
import pandas as pd
train = pd.read_csv("train.csv", sep=',')
test = pd.read_csv("test.csv", sep=',')
train = train.rename(columns={'filenames': 'filename'})
test = test.rename(columns={'filenames': 'filename'})


In [25]:
train.to_csv('train.csv', sep=',', index=False)
test.to_csv('test.csv', sep=',', index=None)

In [26]:
train = pd.read_csv("train.csv", sep=',')
test = pd.read_csv("test.csv", sep=',')
test

,filename,words
0,8000.jpg,sJwнТЌk)ў(ӏ;f_
1,8001.jpg,жX[мъaгМccлёЋaqіДҶU
2,8002.jpg,rЈmјЌaщd№
3,8003.jpg,іаїH@%Uу|қЊ>ПO
4,8004.jpg,Lr}EmөNj*ъ|Ы5UўЏPa!
...,...,...
1995,9995.jpg,tӨВжя.З0ёМUkѓаӏjНdӢ
1996,9996.jpg,ғ}c1$Дѓ#ўC
1997,9997.jpg,qЕ
1998,9998.jpg,k_Ё


In [6]:
df = pd.read_csv("train.csv", sep=',')

In [7]:
df = df.iloc[:80]
df.to_csv("trainsmall.csv", sep=',')

In [8]:
df

,filename,words
0,0.jpg,5Hвkг
1,1.jpg,ӢWвйДfsЯћ
2,2.jpg,?;ГoJю'/ФЩЁ
3,3.jpg,}n&0oZаҚКZUBрѓ}cEnмS
4,4.jpg,Ш)ҚLЯ
...,...,...
75,75.jpg,Хpғ%ЏsжЯќ`
76,76.jpg,gЉЦVү#ӣЉ:қGоaДҷ
77,77.jpg,ЎИbҶяWfC`_Ў
78,78.jpg,оґБ


In [9]:
df = pd.read_csv("test.csv", sep=',')
df = df.iloc[80:100]
df.to_csv("testsmall.csv", sep=',')

In [10]:
df

,filename,words
80,8080.jpg,Itњ9бВ№zҶр`Nnq
81,8081.jpg,@tҷjвЗX(ыөЕ
82,8082.jpg,OиІNљdю
83,8083.jpg,e
84,8084.jpg,ВғШӣ
85,8085.jpg,]ЌZҚ'Кщрө
86,8086.jpg,`ё)уЕуҐ
87,8087.jpg,;КdиӀHmҚ]яYа<ы@w+М
88,8088.jpg,Щ0ҷ1l0ЬСа}Ж
89,8089.jpg,Р
